# Análise de Dados de Mineração Ilegal na Amazônia

In [1]:
# Utilizando uma Base de Dados do RAISG
from owslib.wms import WebMapService

# URL base (sem o request específico)
url = "https://geo.socioambiental.org/webadaptor2/services/PilotoGarimpo/garimpo_ilegal/MapServer/WMSServer"

# Conectando ao serviço WMS
wms = WebMapService(url, version='1.3.0')

# Explorando os Layers
for layer_id, layer in wms.contents.items():
    print(f"{layer_id}: {layer.title}")

2: Bosque Protectores (Ecuador)
3: ANP departamental
4: ANP Nacional
5: TI afectada
6: Minería: concesiones legales
8: áreas de ocurrencia
9: balsas
10: Localización puntual
11: Amazonía
13: Municipal
14: Departamental
15: vias
17: ciudad principal
18: capital de Departamento/Estado
19: capital de país


In [2]:
import geemap

# Criar mapa centralizado na Amazônia
Map = geemap.Map(center=[-4, -62], zoom=5)

Map.add_basemap('HYBRID')  # Imagem de satélite do Google com rótulos
Map.add_basemap('Esri.WorldImagery')  # Outro visual bonito

# Adicionar camada de áreas de ocorrência (layer 8)
Map.add_wms_layer(
    url="https://geo.socioambiental.org/webadaptor2/services/PilotoGarimpo/garimpo_ilegal/MapServer/WMSServer",
    layers="8",
    name="Áreas de Ocorrência",
    format="image/png",
    transparent=True
)

# Exibir o mapa
Map



Successfully saved authorization token.


Map(center=[-4, -62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

---

## 🧱 CONSTRUÇÃO DO CONJUNTO DE DADOS

Nesta etapa, vamos construir um conjunto de dados georreferenciado contendo amostras de regiões com e sem ocorrência de **mineração ilegal**.

### 🎯 Objetivo

Extrair imagens de satélite de alta resolução que serão utilizadas como entradas para modelos de classificação. As imagens serão organizadas em duas categorias:

- `garimpo`: áreas com ocorrência conhecida de mineração ilegal;
- `controle`: áreas sem sinais de mineração, usadas como referência.

---

### 🛰️ Fonte dos dados

Utilizaremos imagens do satélite **Sentinel-2**, da coleção `COPERNICUS/S2_SR_HARMONIZED`, disponível no **Google Earth Engine**. Essa coleção oferece:

- Resolução espacial de até **10 metros**;
- Cobertura global;
- Imagens com bandas multiespectrais;
- Atualizações frequentes (a cada 5 dias em média).

---

### ⚙️ Processo de coleta

1. **Selecionamos pontos de amostragem geográficos** com base em shapefiles:
   - Polígonos de áreas afetadas (`MineriaIlegal_pol.shp`);
   - Pontos pontuais registrados (`MineriaIlegal_pt.shp`);
   - Pontos aleatórios fora dessas áreas (classe controle).

2. **Para cada ponto:**
   - Selecionamos a **imagem mais recente com menor cobertura de nuvens** disponível na coleção Sentinel-2;
   - Recortamos um patch (ex: 512m x 512m) centrado no ponto;
   - Extraímos as bandas visuais (`B4`, `B3`, `B2`, correspondentes a RGB);
   - Salvamos o patch como imagem `.png`.

---

### 📂 Estrutura do diretório de saída

As imagens são salvas automaticamente na seguinte estrutura de diretórios:

```
dataset/
├── garimpo/
│   ├── img_0000.png
│   ├── img_0001.png
│   └── ...
├── controle/
│   ├── img_0000.png
│   ├── img_0001.png
│   └── ...
```

---

### ✅ Resultado final

Um conjunto de dados visual, padronizado, e pronto para ser utilizado no treinamento de modelos de **classificação supervisionada**, como CNNs no TensorFlow.

---

In [5]:
from utils.sample_collector import generate_samples_and_download_images

samples = generate_samples_and_download_images(
    shp_pol_path="MineriaIlegal_2022/MineriaIlegal_pol.shp",
    shp_pt_path="MineriaIlegal_2022/MineriaIlegal_pt.shp",
    n_samples_mining=10,
    n_samples_control=10,
    dataset_dir="dataset"
)

Saving to: dataset\mining\img_0000.png


img_0000.png: |          | 0.00/16.5k (raw) [  0.0%] in 00:00 (eta:     ?)

Saving to: dataset\mining\img_0001.png


img_0001.png: |          | 0.00/16.2k (raw) [  0.0%] in 00:00 (eta:     ?)

Saving to: dataset\mining\img_0002.png


...

Saving to: dataset\control\img_0008.png


img_0008.png: |          | 0.00/16.2k (raw) [  0.0%] in 00:00 (eta:     ?)

Saving to: dataset\control\img_0009.png


img_0009.png: |          | 0.00/16.2k (raw) [  0.0%] in 00:00 (eta:     ?)


✅ Samples saved: {'mining': 10, 'control': 10}
📂 Folder: dataset
